# T1547.012 - Print Processors
Adversaries may abuse print processors to run malicious DLLs during system boot for persistence and/or privilege escalation. Print processors are DLLs that are loaded by the print spooler service, `spoolsv.exe`, during boot.(Citation: Microsoft Intro Print Processors)

Adversaries may abuse the print spooler service by adding print processors that load malicious DLLs at startup. A print processor can be installed through the <code>AddPrintProcessor</code> API call with an account that has <code>SeLoadDriverPrivilege</code> enabled. Alternatively, a print processor can be registered to the print spooler service by adding the <code>HKLM\SYSTEM\\[CurrentControlSet or ControlSet001]\Control\Print\Environments\\[Windows architecture: e.g., Windows x64]\Print Processors\\[user defined]\Driver</code> Registry key that points to the DLL.

For the malicious print processor to be correctly installed, the payload must be located in the dedicated system print-processor directory, that can be found with the <code>GetPrintProcessorDirectory</code> API call, or referenced via a relative path from this directory.(Citation: Microsoft AddPrintProcessor May 2018) After the print processors are installed, the print spooler service, which starts during boot, must be restarted in order for them to run.(Citation: ESET PipeMon May 2020)

The print spooler service runs under SYSTEM level permissions, therefore print processors installed by an adversary may run under elevated privileges.

## Atomic Tests

### Atomic Test #1 - Print ProcessorsEstablishes persistence by creating a new print processor registry key under HKEY_LOCAL_MACHINE\SYSTEM\CurrentControlSet\Control\Print\Environments\Windows x64\Print Processors.
The new print processor will point to a DLL which will be loaded by the spooler service after a reboot. The DLL will then create the file AtomicTest.txt in C:\Users\Public\ as validation that the test is successful.

Note: The test assumes a x64 Windows operating system.

The payload source code is based on a blog post by stmxcsr: [https://stmxcsr.com/persistence/print-processor.html](https://stmxcsr.com/persistence/print-processor.html)
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
if( $(get-service -Name spooler).StartType -eq "Disabled") {Set-Service -Name "spooler" -StartupType Automatic}
net stop spooler
Copy-Item "$PathToAtomicsFolder\T1547.012\bin\AtomicTest.dll" C:\Windows\System32\spool\prtprocs\x64\AtomicTest.dll
reg add "HKLM\SYSTEM\CurrentControlSet\Control\Print\Environments\Windows x64\Print Processors\AtomicRedTeam" /v "Driver" /d "AtomicTest.dll" /t REG_SZ /f
net start spooler
if(0){
  Restart-Computer
}
```

In [ ]:
Invoke-AtomicTest T1547.012 -TestNumbers 1

#### Cleanup: 
```powershell
net stop spooler
rm -force C:\Windows\System32\spool\prtprocs\x64\AtomicTest.dll -ErrorAction SilentlyContinue
rm -force C:\Users\Public\AtomicTest.txt -ErrorAction SilentlyContinue
remove-item "HKLM:\SYSTEM\CurrentControlSet\Control\Print\Environments\Windows x64\Print Processors\AtomicRedTeam" -Force -ErrorAction SilentlyContinue
net start spooler
```

In [ ]:
Invoke-AtomicTest T1547.012 -TestNumbers 1 -Cleanup

## Detection
Monitor process API calls to <code>AddPrintProcessor</code> and <code>GetPrintProcessorDirectory</code>. New print processor DLLs are written to the print processor directory. Also monitor Registry writes to <code>HKLM\SYSTEM\ControlSet001\Control\Print\Environments\\[Windows architecture]\Print Processors\\[user defined]\\Driver</code> or <code>HKLM\SYSTEM\CurrentControlSet\Control\Print\Environments\\[Windows architecture]\Print Processors\\[user defined]\Driver</code> as they pertain to print processor installations.

Monitor for abnormal DLLs that are loaded by spoolsv.exe. Print processors that do not correlate with known good software or patching may be suspicious.